# 本节内容摘要
各种求导运算：  
- 向量链式法则（复合函数求导）  
> 例1（标量对向量求导）  
例2（涉及到矩阵的情况）  
> 
- 自动求导  
> 由于在神经网络动辄几百层，手动进行链式求导是很困难的，因此我们需要借助自动求导  
2.1 计算图内部原理  
2.2 自动求导的两种模式（正向累积和反向累积）  
2.3 复杂度比较
> 

# 讲义

In [1]:
from IPython.display import display, IFrame

# 使用IFrame将PDF文件嵌入到Notebook中
pdf_path = "../PPT/part-0_7.pdf"
display(IFrame(pdf_path, width=1100, height=600))


# 自动微分

假设我们想对函数$y=2\mathbf{x}^{\top}\mathbf{x}$关于列向量$\mathbf{x}$求导

In [2]:
import torch

x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

在我们计算$y$关于$\mathbf{x}$的梯度之前，需要一个地方来存储梯度

In [3]:
x.requires_grad_(True) # 等价于 x = torch.arange(4.0,requires_grad=True)
print(x.grad) # 默认为None

None


现在计算$y$

In [4]:
y = 2 * torch.dot(x, x) # 点积 = 0^2 + 1^2 + 2^2 + 3^2 = 0 + 1 + 4 + 9 = 14----->y=2*x^2
y

tensor(28., grad_fn=<MulBackward0>)

通过调用反向传播函数来自动计算`y`关于`x`每个分量的梯度

In [5]:
y.backward() # 4*x
x.grad

tensor([ 0.,  4.,  8., 12.])

In [6]:
x.grad == 4 * x

tensor([True, True, True, True])

现在计算`x`的另一个函数

In [7]:
# 默认情况下，PyTorch会累积梯度，需要清除之前的值
x.grad.zero_() # y.backward() 后才能产生梯度，才能梯度清零，没有反向传播，无法梯度清零
y = x.sum() # 这里的y是一个标量，sum函数其实就是x_1+x_2+...x_n，求偏导自然是全1啊
y.backward()
x.grad

tensor([1., 1., 1., 1.])

深度学习中
，我们的目的不是计算微分矩阵，而是单独计算批量中每个样本的偏导数之和

In [8]:
# 默认情况下，PyTorch会累积梯度，需要清除之前的值
# 对非标量调用 'backward' 需要传入一个 'gradient' 参数，该参数指定微分函数
x.grad.zero_()
y = x * x  # 这里的y不是一个标量，这是一个向量
print(y)
# 等价于y.backward(torch.ones(len(x)))
y.sum().backward() # y.sum()后将向量转为标量了，对标量求导
x.grad

tensor([0., 1., 4., 9.], grad_fn=<MulBackward0>)


tensor([0., 2., 4., 6.])

将某些计算移动到记录的计算图之外

In [9]:
x.grad.zero_()
y = x * x
u = y.detach() # y.detach把y当作一个常数，而不是关于x的一个函数
z = u * x

z.sum().backward()
x.grad == u

tensor([True, True, True, True])

In [10]:
x.grad.zero_()
y.sum().backward()
x.grad == 2 * x

tensor([True, True, True, True])

即使构建函数的计算图需要通过Python控制流（例如，条件、循环或任意函数调用），我们仍然可以计算得到的变量的梯度

In [11]:
def f(a):
    b = a * 2
    while b.norm() < 1000: # norm是L2范数
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

a = torch.randn(size=(), requires_grad=True)
print(a)

d = f(a)
d.backward()
# 我们现在可以分析上面定义的f函数。 请注意，它在其输入a中是分段线性的。 
# 换言之，对于任何a，存在某个常量标量k，使得f(a)=k*a，其中k的值取决于输入a，因此可以用d/a验证梯度是否正确。
a.grad == d / a # d是a的线性函数，所以导数就是斜率d/a

tensor(1.4540, requires_grad=True)


tensor(True)

这也是隐式构造的优势，因为它会存储梯度计算的计算图，再次计算时执行反向过程就可以